# Baseline

오차행렬 : [731 253 254 10622]


f1 score : 0.7425088872595302

In [232]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### import 함수

In [233]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings(action = 'ignore')

In [234]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

In [235]:
df_train = pd.read_csv('느그들 구글 드라이브 경로에서 복사해서 써라 아니면 오류뜬다 병시들아')
df_test = pd.read_csv('느그들 구글 드라이브 경로에서 복사해서 써라 아니면 오류뜬다 병시들아')

In [236]:
df_train.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,0.25,/ Asansol / India,AS,NaN,47466,End Customer,Enterprise,19.0,NaN,NaN,...,LGEIL,NaN,0,0,NaN,NaN,NaN,NaN,115,False
1,1.00,/Asansol/India,ID,NaN,47466,NaN,Enterprise,4.0,NaN,NaN,...,LGEIL,3 months ~ 6 months,0,0,NaN,NaN,NaN,NaN,175,False
2,0.50,//,ID,NaN,19504,NaN,Enterprise,NaN,NaN,NaN,...,LGEUS,more than a year,0,0,NaN,NaN,NaN,NaN,194,False
3,1.00,/Bangladesh/Pakistan,AS,0.048544,1944,NaN,Enterprise,NaN,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000543,0.022634,special purpose,NaN,68,False
4,0.50,//Germany,IT,NaN,27783,NaN,SMB,NaN,NaN,NaN,...,LGEDG,NaN,0,0,NaN,NaN,NaN,NaN,980,False


### 전처리

In [237]:
df_train.fillna(0, inplace = True)
df_test.fillna(0, inplace = True)

df_train.isna().sum()

bant_submit                0
customer_country           0
business_unit              0
com_reg_ver_win_rate       0
customer_idx               0
customer_type              0
enterprise                 0
historical_existing_cnt    0
id_strategic_ver           0
it_strategic_ver           0
idit_strategic_ver         0
customer_job               0
lead_desc_length           0
inquiry_type               0
product_category           0
product_subcategory        0
product_modelname          0
customer_country.1         0
customer_position          0
response_corporate         0
expected_timeline          0
ver_cus                    0
ver_pro                    0
ver_win_rate_x             0
ver_win_ratio_per_bu       0
business_area              0
business_subarea           0
lead_owner                 0
is_converted               0
dtype: int64

### 모델링

In [238]:
df_test = df_test.drop('is_converted', axis = 1)

In [239]:
def label_encoding(series):
    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

#레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    'id_strategic_ver',
    'it_strategic_ver',
    'business_unit'
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

for col in label_columns:
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]



x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.3,
    shuffle=True,
    random_state=400,
)

In [240]:
model = DecisionTreeClassifier()

model.fit(x_train, y_train)

pred = model.predict(x_val)

In [241]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

get_clf_eval(y_val, pred)

오차행렬:
 [[  876   266]
 [  297 12793]]

정확도: 0.9604
정밀도: 0.7468
재현율: 0.7671
F1: 0.7568


In [242]:
final = model.predict(df_test)

final = pd.DataFrame(final, columns = ['is_converted'])

### 점수 확인

[건들면 오류남 성중이의 노고를 생각해주시길]
* answer 불러오는 경로는 수정하세요

In [244]:
answer = pd.read_csv('느그들 구글 드라이브 경로에서 복사해서 써라 아니면 오류뜬다 병시들아')

def f1_score(final,answer):
  TP = 0
  FP = 0
  FN = 0
  TN = 0

  for i in range(final.shape[0]):
    if (final['is_converted'][i] == True) and (answer['is_converted'][i] == True):
      TP +=1
    elif (final['is_converted'][i] == True) and (answer['is_converted'][i] == False):
      FP +=1
    elif (final['is_converted'][i] == False) and (answer['is_converted'][i] == True):
      FN +=1
    else:
      TN +=1

  precision = TP / (TP + FP + 1e-9)  # 1e-9를 더하여 0으로 나누는 경우를 방지합니다.
  recall = TP / (TP + FN + 1e-9)
  f1_score = 2 * (precision * recall) / (precision + recall + 1e-9)
  return f1_score

# df_test에 너네가 예측한 데이터프레임 형태로 넣으면 된단다.
f1_score(final,answer)

0.7425088872595302